In [ ]:
--as Aleem, I will be ensuring that the first 6 tables are all prepped and created as ###_cleaned tables, ready for use

--1
select * from CUSTOMERS_RAW
--this table looks alright, so we can straightaway rename it as 'CUSTOMERS_CLEANED'.
ALTER TABLE CUSTOMERS_RAW RENAME TO CUSTOMERS_CLEANED;
select * from customers_cleaned  --this is the final customer table

--2
select * from employee_raw
ALTER TABLE EMPLOYEE_RAW RENAME TO EMPLOYEE_CLEANED;
select * from employee_cleaned   --this is the final employee table


--3


select * from inventory_transactions_raw

-- Step 1: Create a cleaned version of the table without unnecessary columns
CREATE OR REPLACE TABLE INVENTORY_TRANSACTIONS_CLEANED AS
SELECT
  TransactionID,
  ProductID,
  CAST(COALESCE(NULLIF(PurchaseOrderID, ''), -1) AS INT) AS PurchaseOrderID,
  COALESCE(
    TransactionDate,
    TO_DATE ('1900-01-01', 'yyyy-mm-DD')
  ) AS TransactionDate,
  CAST(
    COALESCE(NULLIF(UnitPurchasePrice, ''), 0) AS FLOAT
  ) AS UnitPurchasePrice,
  CAST(COALESCE(NULLIF(QuantityOrdered, ''), 0) AS INT) AS QuantityOrdered,
  CAST(COALESCE(NULLIF(QuantityReceived, ''), 0) AS INT) AS QuantityReceived,
  CAST(COALESCE(NULLIF(QuantityMissing, ''), 0) AS INT) AS QuantityMissing
FROM
  INVENTORY_TRANSACTIONS_RAW
WHERE
  NOT (MissingID IS NULL);



-- Step 2: Verify the cleaned data
SELECT *
FROM INVENTORY_TRANSACTIONS_CLEANED
LIMIT 10;


-- Step 3: Add indexes or constraints (optional)
-- For example, if TransactionID should be unique:
ALTER TABLE INVENTORY_TRANSACTIONS_CLEANED ADD PRIMARY KEY (TransactionID);  --this is the final inventory transactions ID although there are still issues to be fixed.

SELECT * FROM INVENTORY_TRANSACTIONS_CLEANED;


--4

select * from orders_raw


-- Step 1: Create a cleaned version of the table
CREATE OR REPLACE TABLE ORDERS_CLEANED AS
SELECT
    ORDERID,
    CUSTOMERID,
    EMPLOYEEID,
    -- Replace NULL or empty strings with 0 and cast to INTEGER to remove decimal points
    CAST(COALESCE(NULLIF(SHIPPINGMETHODID, ''), 0) AS INTEGER) AS SHIPPINGMETHODID,
    ORDERDATE,
    SHIPDATE,
    FREIGHTCHARGE
FROM ORDERS_RAW;

-- Step 2: Verify the cleaned table
SELECT *
FROM ORDERS_CLEANED


-- this is the final orders table


select * from order_details_raw
ALTER TABLE order_details_raw RENAME TO order_details_CLEANED;
select * from order_details_cleaned  --this is the final order details table


--5

select * from payments_raw

-- Step 1: Create a cleaned version of the table without rows containing missing or blank values
CREATE OR REPLACE TABLE PAYMENTS_CLEANED AS
SELECT
    PAYMENTID,
    ORDERID,
    PAYMENTMETHODID,
    PAYMENTDATE,
    PAYMENTAMOUNT AS DECIMAL
FROM PAYMENTS_RAW
WHERE PAYMENTID IS NOT NULL
  AND ORDERID IS NOT NULL
  AND PAYMENTMETHODID IS NOT NULL AND PAYMENTMETHODID != ''
  AND PAYMENTDATE IS NOT NULL 
  AND PAYMENTAMOUNT IS NOT NULL

-- Step 2: Verify the cleaned table
SELECT *
FROM PAYMENTS_CLEANED